In [1]:
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint
from langchain.prompts import ChatPromptTemplate
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

In [2]:
from langchain_community.document_loaders import WebBaseLoader

    loader = WebBaseLoader(
        "https://blog.langchain.dev/langchain-v0-1-0/"
    )
    
    documents = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

In [4]:
embeddings = FastEmbedEmbeddings()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

In [5]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

In [6]:
retriever = vector_store.as_retriever()

In [7]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3", format="json", temperature=0)



In [8]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [9]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context")}
)

In [10]:
question = "What are the major changes in v0.1.0?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})


In [11]:
result

{'response': AIMessage(content='{"Any breaking changes to the public API will result in a minor version bump (the second digit)Any bug fixes or new features will result in a patch version bump (the third digit)" \n\n: "LangChain has been around for a little over a year and has changed a lot as it‚Äôs grown to become the default framework for building LLM applications. As we previewed a month ago, we recently decided to make significant changes to the" \n\n}', response_metadata={'model': 'llama3', 'created_at': '2024-07-30T08:34:21.97810824Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 36333348481, 'load_duration': 16444955, 'prompt_eval_count': 850, 'prompt_eval_duration': 13012807000, 'eval_count': 96, 'eval_duration': 23007663000}, id='run-1ac3ebf9-4371-4bae-82f9-1aaeb0d69cdd-0'),
 'context': [Document(page_content='0 created a few challenges:Users couldn‚Äôt be confident that updating would not have breaking changeslangcha